# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd
from IPython.core.display import HTML
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('homepage_actions.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [2]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
print(df.action.unique())
print(df.group.unique())

['view' 'click']
['experiment' 'control']


In [4]:
from pandasql import sqldf

def pysqldf(query_string):
    return sqldf(query_string, globals())

In [5]:
df_viewers = pysqldf("SELECT DISTINCT id FROM df WHERE action='view'")
print(f"{len(df_viewers)} users viewed.")

6328 users viewed.


### How many viewers also clicked?

In [6]:
df_viewers_that_clicked = pysqldf("""
    SELECT 
        * 
    FROM 
        df 
    WHERE 
        id IN (SELECT id FROM df_viewers) 
    AND 
        action='click'
""")

print(f"{len(df_viewers_that_clicked)} users that viewed also clicked.")

1860 users that viewed also clicked.


### Did anyone click that didn't view?

In [7]:
df_clickers = pysqldf("SELECT DISTINCT id FROM df WHERE action='click'")
print(f"{len(df_clickers)} users clicked.")

1860 users clicked.


In [8]:
df_clickers_no_view = pysqldf("""
    SELECT 
        * 
    FROM 
        df 
    WHERE 
        id IN (SELECT id FROM df_clickers) 
    AND 
        id NOT IN (SELECT id FROM df_viewers)
""")

print(f"{len(df_clickers_no_view)} users clicked but did not view.")

0 users clicked but did not view.


### Did anyone view that didn't click?

In [9]:
df_viewers_no_click = pysqldf("""
    SELECT 
        * 
    FROM 
        df 
    WHERE 
        id IN (SELECT id FROM df_viewers) 
    AND 
        id NOT IN (SELECT id FROM df_clickers)
""")

print(f"{len(df_viewers_no_click)} users viewed but did not click.")

4468 users viewed but did not click.


### Is there any overlap between the control and experiment groups?

In [10]:
df_control = pysqldf("SELECT DISTINCT id FROM df WHERE [group]='control'")
print(f"{len(df_control)} users in control group.")

df_exp = pysqldf("SELECT DISTINCT id FROM df WHERE [group]='experiment'")
print(f"{len(df_exp)} users in experiment group.")

df_group_overlap = pysqldf("""SELECT id FROM df_control JOIN df_exp USING(id)""")
print(f"{len(df_group_overlap)} users in both control AND experiment group.")

3332 users in control group.
2996 users in experiment group.
0 users in both control AND experiment group.


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

### $H_0$ Hypothesis

Qualitative:

$H_0$: The number of users in the experimental group that viewed the homepage AND clicked is less than the number of users in the control group that viewed the homepage AND clicked.

<p><br><br>
Quantitative:

Let $c_e$ denote the number of users in the experimental group that viewed the homepage AND clicked.

Let $c_c$ denote the number of users in the control group that viewed the homepage AND clicked.

$H_0$: $c_e < c_c$.

<p><br><br>

## $H_a$ Hypothesis
Qualitative:

$H_a$: The number of users in the experimental group that viewed the homepage AND clicked is greater than or equal to the number of users in the control group that viewed the homepage AND clicked.

<p><br><br>
Quantitative:

$H_a$: $c_e \ge c_c$.

<p><br><br>

### Thus, based on $H_0$, a one-sided test is required.

In [11]:
df_viewers_that_clicked__control = pysqldf("""SELECT id FROM df_viewers_that_clicked JOIN df_control USING(id)""")
df_control['clicked'] = 0
df_control.loc[df_control['id'].isin(df_viewers_that_clicked__control['id'].values), 'clicked'] = 1
c_clicked = df_control.clicked.sum()
n_c = len(df_control)
c_c = c_clicked/n_c
print(f"click-rate for control group: {c_clicked} out of {n_c} ({c_c})")

click-rate for control group: 932 out of 3332 (0.2797118847539016)


In [12]:
df_viewers_that_clicked__exp = pysqldf("""SELECT id FROM df_viewers_that_clicked JOIN df_exp USING(id)""")
df_exp['clicked'] = 0
df_exp.loc[df_exp['id'].isin(df_viewers_that_clicked__exp['id'].values), 'clicked'] = 1
e_clicked = df_exp.clicked.sum()
n_e = len(df_exp)
c_e = e_clicked/n_e
print(f"click-rate for exp group: {e_clicked} out of {n_e} ({c_e})")

click-rate for exp group: 928 out of 2996 (0.3097463284379172)


In [13]:
import scipy.stats as stats
import numpy as np

def var(x):
    return sum((x - x.mean())**2)/(len(x)-1)

def welch_t(a, b):
    return abs((a.mean() - b.mean())/np.sqrt((var(a)/len(a)) + (var(b)/len(b))))

def welch_df(a, b):
    var_a = var(a)
    len_a = len(a)
    var_b = var(b)
    len_b = len(b)
    return (((var_a/len_a) + (var_b/len_b))**2)/((var_a**2/(len_a**2 * (len_a-1))) + (var_b**2/(len_b**2 * (len_b-1))))

def p_value_welch(a, b, two_sided=False):
    t = welch_t(a, b)
    df = welch_df(a, b)
    #p-val for one-sided t-test:
    p = 1 - stats.t.cdf(t, df) # according to the lecture for this lab, this provides the p-val for a one-sided t-test
    return p*2 if two_sided else p

In [14]:
#Your code here
p_val = p_value_welch(df_exp.clicked, df_control.clicked, two_sided=False)
print(f"p_val is {p_val}")
alpha = 0.05
if p_val < alpha:
    print("Therefore, we can reject the null hypothesis and accept the alternate hypothesis with {(1-alpha)*100}% confidence.")
else:
    print("Therefore, we cannot reject the null hypothesis.")

p_val is 0.004466402814337078
Therefore, we can reject the null hypothesis and accept the alternate hypothesis with {(1-alpha)*100}% confidence.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [15]:
#Your code here
expected_clicks_exp = c_c * n_e
expected_clicks_exp

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [16]:
#Your code here
var_exp = n_e * c_c * (1 - c_c)
sd_exp = np.sqrt(var_exp)
actual_clicks_exp = e_clicked
#z-score measure number of std deviations
z_score_exp_clicks = abs(actual_clicks_exp - expected_clicks_exp)/sd_exp
z_score_exp_clicks

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [17]:
#Your code here
p_val = 1 - stats.norm.cdf(z_score_exp_clicks)
p_val

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

Both methods used to compute p-value produce a result that allows us to reject $H_0$.  Thus, we can say with 95% confidence that the homepage to which the experimental group was subjected is is more effective based on clicks.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.